In [1]:
import sys
print(sys.executable)

/home/matthias/anaconda3/envs/lungscans/bin/python


In [2]:
import os
import torch
import numpy as np

# CODE FILES HERE
from model_params import get_model_data_tdhcvae
from models.tdhcvae.tdhcvae import Tdhcvae
from solver import Solver
from dataloader import DataLoader
from directories import Directories
from plots import plot_losses, plot_gaussian_distributions, plot_rl_kl, plot_latent_space,\
plot_latent_manifold, plot_faces_grid, plot_faces_samples_grid,\
plot_prepro_alpha_params_distribution, plot_prepro_radius_params_distribution,\
plot_transformed_images, plot_y_space_thetas, plot_y_space_scales
from auxiliary import get_latent_spaces, transform_images
from preprocessing import RandomPreprocessing, DeterministicPreprocessing

# SETTINGS HERE
os.environ['CUDA_LAUNCH_BLOCKING'] = "1" # to see the CUDA stack
%matplotlib inline
# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2
# supress cluttering warnings in solutions
import warnings
warnings.filterwarnings('ignore')

In [3]:
# setting device on GPU if available, else CPU
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

# Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_cached(0)/1024**3,1), 'GB')

Using device: cpu



In [4]:
dataset_arg = "lungscans"
data = get_model_data_tdhcvae(dataset_arg)

In [5]:
directories = Directories("tdcvae", dataset_arg.lower(), data["z_dim"], make_dirs=False)
data_loader = DataLoader(directories, data["batch_size"], dataset_arg.lower(), resize=data["resize"])
model = Tdhcvae(data["z_dim"], data["beta"])
solver = Solver(model, data_loader, data["optimizer"], data["epochs"], data["optim_config"], step_config=data["step_config"],\
                lr_scheduler=data["lr_scheduler"], tdhcvae_mode=True, save_model_state=False)
solver.main()

hihii
+++++ START RUN +++++ | no save mode
0 1
(64, 64)
torch.Size([1, 1, 64, 64]) torch.Size([1, 1, 64, 64]) 4096
her torch.Size([1, 1, 64, 64])


RuntimeError: size mismatch, m1: [64 x 64], m2: [4096 x 1024] at /opt/conda/conda-bld/pytorch_1556653183467/work/aten/src/TH/generic/THTensorMath.cpp:961

In [ ]:
#Uncomment to load a model to continue training.
#solver = torch.load("../results/tdcvae/MNIST_z=2_0/model_state.pt")
#solver.main()

In [ ]:
#Uncomment to load a trained model for inference.
#solver = torch.load("../../result_instance_2/tdcvae/MNIST_z=2_12/model_state.pt", map_location='cpu')
#solver.model.eval()
#solver.data_loader.directories.make_dirs = False

In [ ]:
# Plotting train and test losses for all epochs
plot_losses(solver, solver.train_loss_history["train_loss_acc"], solver.test_loss_history)

In [ ]:
# Plotting the gaussian of z space and some metrics about the space
plot_gaussian_distributions(solver, len(solver.train_loss_history["train_loss_acc"]))

In [ ]:
# Monitoring the reconstruction loss (likelihood lower bound) and KL divergence
DEBUG = 1
if DEBUG:
    for epoch, train_loss, test_loss, rl, kl in zip(solver.train_loss_history["epochs"], \
        solver.train_loss_history["train_loss_acc"], solver.test_loss_history, \
        solver.train_loss_history["recon_loss_acc"], solver.train_loss_history["kl_diverg_acc"]):
        print("epoch: {}, train_loss: {:.2f}, test_loss: {:.2f}, recon. loss: {:.2f}, KL div.: {:.2f}".format(
            epoch, train_loss, test_loss, rl, kl))
plot_rl_kl(solver, solver.train_loss_history["recon_loss_acc"], solver.train_loss_history["kl_diverg_acc"])

In [ ]:
last_train_loss = solver.train_loss_history["train_loss_acc"][-1]
mode = ""
if solver.data_loader.directories.make_dirs:
    if solver.data_loader.thetas and solver.data_loader.scales:
        mode = "SCALES_THETAS_"
    elif solver.data_loader.thetas:
        mode += "THETAS_"
    elif solver.data_loader.scales:
        mode += "SCALES_"
torch.save(solver, solver.data_loader.directories.result_dir + "/model_TD_CVAE_" + mode + solver.data_loader.dataset + "_train_loss=" + "{0:.2f}".format(last_train_loss) + "_z=" + str(solver.model.z_dim) + ".pt")